In [2]:
import xarray as xr
import rioxarray as rxr
from dask.diagnostics import ProgressBar

da = rxr.open_rasterio('../data/CombinedData_32signed/CombinedData32signed.tif')

ds = xr.Dataset(
    {
        'rs': da.sel(band=[1, 2, 3, 4]),
        'label': da.sel(band=5),
        'mask': da.sel(band=6),
    }
)

ds = ds.transpose('band', 'x', 'y')

category_dict_reversed = {
    0: "ConstructionSite",
    1: "Building",
    2: "BuildingDistortion",
    3: "GreenAreas",
    4: "RoadAsphalt",
    5: "Forest",
    6: "WaterBasin",
    7: "Path",
    8: "MeadowPasture",
    9: "SealedObjects"
}

# Dataset attributes
ds.attrs = {'creator': 'Julian Kraft'}

# Variable attributes
ds['rs'].attrs.update({'source': 'SwissImage RS'})
ds['label'].attrs.update({'classes': ', '.join([f'{k}={v}' for k, v in category_dict_reversed.items()])})

ds['rs'] = ds.rs.chunk({'band': 4, 'x': 100, 'y': 100})
ds['label'] = ds.label.chunk({'x': 100, 'y': 100})
ds['mask'] = ds.mask.chunk({'x': -1, 'y': -1})


encoding = {}

for variable in ds.data_vars:
    encoding[variable] = {'compressor': None}

    if variable == 'rs':
        ds[variable] = ds[variable].astype('int')

    else:
        ds[variable] = ds[variable].where(ds[variable] != -9999, -1).astype('int8')

        if variable == 'mask':
            chunks = [-1, -1]
        elif variable == 'label':
            chunks = [100, 100]

with ProgressBar():
    ds.to_zarr('../data/combined.zarr', encoding=encoding, mode='w')

/Users/kraftb/miniforge3/envs/machflow/lib/python3.10/site-packages/pyproj/crs/_cf1x8.py:514: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


[#########                               ] | 24% Completed | 163.07 s

: 

In [ ]:
ds = xr.open_zarr('../data/combined.zarr')